In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [2]:
def merge_csv_folder(path):
    """
    path에 있는 모든 csv 파일을 읽어서 하나의 데이터프레임으로 합친다.
    """
    dfs = []
    for root, dirs, files in os.walk(path):
        for file_name in files:
            if not file_name.endswith('.csv'):
                continue
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            dfs.append(df)
    if len(dfs) == 0:
        return None
    return pd.concat(dfs, ignore_index=True)

# 사이트별로 데이터를 합치기
사이트_목록 = ['bobae', 'clien', 'fmkorea', 'naver_cafe']
사건_목록 = ["벤츠 화재", "아이오닉 누수", "아이오닉 iccu", "코나 화재"]
게시글_댓글 = ["comments", "posts"]

# 수, 사건, 사이트, 종류 를 컬럼으로 가지는 데이터 프레임
df_table = pd.DataFrame(columns=["수", "사건", "사이트", "종류"])


"벤츠 화재/bobae/comments"
"사건/사이트/게시글_댓글"
for 사건 in 사건_목록:
    for 사이트 in 사이트_목록:
        if 사이트 != "naver_cafe":
            for 종류 in 게시글_댓글:
                df = merge_csv_folder(f"{사건}/{사이트}/{종류}")
                if df is None:
                    continue
                # 새로운 데이터를 딕셔너리 형태로 생성
                new_row = {"수": len(df), "사건": 사건, "사이트": 사이트, "종류": 종류}

                # 딕셔너리를 DataFrame으로 변환한 뒤 기존 DataFrame과 병합
                df_table = pd.concat([df_table, pd.DataFrame([new_row])], ignore_index=True)
        elif 사이트 == "naver_cafe":
            # f"{사건}/{사이트}/" 디렉토리 내부의 모든 폴더의 path를 가져옴
            for root, dirs, files in os.walk(f"{사건}/{사이트}/"):
                for dir in dirs:
                    for 종류 in 게시글_댓글:
                        df = merge_csv_folder(f"{사건}/{사이트}/{dir}/{종류}")
                        if df is None:
                            continue
                        # 새로운 데이터를 딕셔너리 형태로 생성
                        new_row = {"수": len(df), "사건": 사건, "사이트": 사이트, "종류": 종류}

                        # 딕셔너리를 DataFrame으로 변환한 뒤 기존 DataFrame과 병합
                        df_table = pd.concat([df_table, pd.DataFrame([new_row])], ignore_index=True)
            
            
# df_table에서 사건 사이트 별로 수를 합산
df_table_sum = df_table.groupby(["사건", "사이트", "종류"]).sum().reset_index()

# 사이트 순서로 정렬
df_table_sum = df_table_sum.sort_values(by=["사이트", "사건", "종류"], ascending=True)
df_table_sum

# 사건, 사이트 별로 수를 더한 행을 추가
df_table_sum_both = df_table_sum.groupby(["사건", "사이트"]).sum().reset_index()

# df_table_sum_both의 "종류"를 _both로 변경
df_table_sum_both["종류"] = "합계"

# df_table_sum에 df_table_sum_both 를 concat
df_table_sum = pd.concat([df_table_sum, df_table_sum_both], ignore_index=True)
df_table_sum = df_table_sum.sort_values(by=["사이트", "사건", "종류"], ascending=True)
df_table_sum

# 사이트 컬럼을 0번 컬럼으로 이동
df_table_sum = df_table_sum[["사이트", "사건", "종류", "수"]]
df_table_sum





,사이트,사건,종류,수
0,bobae,벤츠 화재,comments,96
1,bobae,벤츠 화재,posts,112
32,bobae,벤츠 화재,합계,208
2,bobae,아이오닉 iccu,comments,12
3,bobae,아이오닉 iccu,posts,16
36,bobae,아이오닉 iccu,합계,28
4,bobae,아이오닉 누수,comments,11
5,bobae,아이오닉 누수,posts,15
40,bobae,아이오닉 누수,합계,26
6,bobae,코나 화재,comments,202


In [3]:
# df_table_sum 의 사이트, 종류 별로 그룹화하여 수의 합을 구함
df_table_sum_pivot = df_table_sum.pivot_table(index=["종류"], columns="사건", values="수", aggfunc="sum", fill_value=0)
df_table_sum_pivot

/var/folders/v4/6tjs7h4921v9vrm_844m1clw0000gn/T/ipykernel_49745/1026153409.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_table_sum_pivot = df_table_sum.pivot_table(index=["종류"], columns="사건", values="수", aggfunc="sum", fill_value=0)


사건,벤츠 화재,아이오닉 iccu,아이오닉 누수,코나 화재
종류,,,,
comments,15955,24444,3258,54080
posts,1093,1403,279,3517
합계,17048,25847,3537,57597


In [4]:
# df_table_sum_pivot 1번째 컬럼은 그대로 놓고 나머지 모든 컬럼을 합한 컬럼만 남겨 다른 데이터 프레임에 저장
df_table_sum_pivot_sum = df_table_sum_pivot.copy()
df_table_sum_pivot_sum["합계"] = df_table_sum_pivot_sum.sum(axis=1)
df_table_sum_pivot_sum = df_table_sum_pivot_sum[["합계"]]
df_table_sum_pivot_sum

사건,합계
종류,
comments,97737
posts,6292
합계,104029


In [5]:
df = pd.DataFrame(df_table_sum)

# 사건 및 사이트별로 크로스탭 생성
crosstab = pd.pivot_table(df, index=["사건", "종류"], columns=["사이트"], values="수", aggfunc='sum', fill_value=0)

# 행과 열 이름을 적절하게 변경
crosstab.columns = [f"{site}" for site in crosstab.columns]

# 예쁘게 출력
crosstab


# '종류'열의 값이 '합계'인 행들만 모두 더해서 '총합계' 행을 생성
crosstab.loc[("총합계", "합계"), :] = crosstab.sum(axis=0) // 2
crosstab    

/var/folders/v4/6tjs7h4921v9vrm_844m1clw0000gn/T/ipykernel_49745/860001287.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  crosstab = pd.pivot_table(df, index=["사건", "종류"], columns=["사이트"], values="수", aggfunc='sum', fill_value=0)


bobae    clien  fmkorea  naver_cafe
사건        종류                                           
벤츠 화재     comments   96.0   1545.0   1624.0     12690.0
          posts     112.0     58.0    165.0       758.0
          합계        208.0   1603.0   1789.0     13448.0
아이오닉 iccu comments   12.0   8176.0    545.0     15711.0
          posts      16.0    295.0     61.0      1031.0
          합계         28.0   8471.0    606.0     16742.0
아이오닉 누수   comments   11.0    375.0    147.0      2725.0
          posts      15.0     10.0     18.0       236.0
          합계         26.0    385.0    165.0      2961.0
코나 화재     comments  202.0  13105.0    349.0     40424.0
          posts     329.0    514.0     31.0      2643.0
          합계        531.0  13619.0    380.0     43067.0
총합계       합계        793.0  24078.0   2940.0     76218.0